In [1]:
# import necessary libs
import numpy  as np
import pandas as pd

# general spark modules
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType #https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html


In [2]:
# load data as dataframe
df = spark.read.csv('/FileStore/tables/Fifa_2017-ec2ed.csv', header=True)

In [3]:
df.show(10)

+---+------+-----------------+---+--------------------+-----------+-------+---------+-------------------+-------+-----+--------------+---------+-----------+--------+-------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+-------+--------+---------+-------------+---------+--------------+--------+----------+---------+-------------+----------+--------------+
_c0| ID| Name|Age| Photo|Nationality|Overall|Potential| Club| Value| Wage|Preferred Foot|Weak Foot|Skill Moves|Position|Jersey Number|Height|Weight| LS| ST| RS| LW| RW| CAM| LM| CM| RM| CDM| LB| LCB| CB| RCB| RB|Crossing|Finishing|HeadingAccuracy|ShortPassing|Volleys|Dribbling|Curve|FKAccuracy|LongPassing|BallControl|Acceleration|SprintSpeed|Balance|ShotPower|Jumping|Stamina|Strength|LongShots|Interceptions|Penalties|StandingTackle|GKDiving|GKHandling|GKKicking|GKPositioning|GKReflexes|Release Clause|
+---+------+-----------------+---+--------------------+-----------+-------+---------+-------------------+-------+-----+--------------+---------+-----------+--------+-------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+-------+---------+-------+-------+--------+---------+-------------+---------+--------------+--------+----------+---------+-------------+----------+--------------+
 0|158023| L. Messi| 31|https://cdn.sofif...| Argentina| 94| 94| FC Barcelona|€110.5M|€565K| Left| 4| 4| RF| 10| 5'7|159lbs|88+2|88+2|88+2|92+2|92+2|93+2|91+2|84+2|91+2|61+2|59+2|47+2|47+2|47+2|59+2| 84| 95| 70| 90| 86| 97| 93| 94| 87| 96| 91| 86| 95| 85| 68| 72| 59| 94| 22| 75| 28| 6| 11| 15| 14| 8| €226.5M|
 1| 20801|Cristiano Ronaldo| 33|https://cdn.sofif...| Portugal| 94| 94| Juventus| €77M|€405K| Right| 4| 5| ST| 7| 6'2|183lbs|91+3|91+3|91+3|89+3|89+3|88+3|88+3|81+3|88+3|61+3|61+3|53+3|53+3|53+3|61+3| 84| 94| 89| 81| 87| 88| 81| 76| 77| 94| 89| 91| 70| 95| 95| 88| 79| 93| 29| 85| 31| 7| 11| 15| 14| 11| €127.1M|
 2|190871| Neymar Jr| 26|https://cdn.sofif...| Brazil| 92| 93|Paris Saint-Germain|€118.5M|€290K| Right| 5| 5| LW| 10| 5'9|150lbs|84+3|84+3|84+3|89+3|89+3|89+3|88+3|81+3|88+3|60+3|60+3|47+3|47+3|47+3|60+3| 79| 87| 62| 84| 84| 96| 88| 87| 78| 95| 94| 90| 84| 80| 61| 81| 49| 82| 36| 81| 24| 9| 9| 15| 15| 11| €228.1M|
 3|193080| De Gea| 27|https://cdn.sofif...| Spain| 91| 93| Manchester United| €72M|€260K| Right| 3| 1| GK| 1| 6'4|168lbs|null|null|null|null|null|null|null|null|null|null|null|null|null|null|null| 17| 13| 21| 50| 13| 18| 21| 19| 51| 42| 57| 58| 43| 31| 67| 43| 64| 12| 30| 40| 21| 90| 85| 87| 88| 94| €138.6M|
 4|192985| K. De Bruyne| 27|https://cdn.sofif...| Belgium| 91| 92| Manchester City| €102M|€355K| Right| 5| 4| RCM| 7| 5'11|154lbs|82+3|82+3|82+3|87+3|87+3|88+3|88+3|87+3|88+3|77+3|73+3|66+3|66+3|66+3|73+3| 93| 82| 55| 92| 82| 86| 85| 83| 91| 91| 78| 76| 77| 91| 63| 90| 75| 91| 61| 79| 58| 15| 13| 5| 10| 13| €196.4M|
 5|183277| E. Hazard| 27|https://cdn.sofif...| Belgium| 91| 91| Chelsea| €93M|€340K| Right| 4| 4| LF| 10| 5'8|163lbs|83+3|83+3|83+3|89+3|89+3|89+3|89+3|82+3|89+3|63+3|60+3|49+3|49+3|49+3|60+3| 81| 84| 61| 89| 80| 95| 83| 79| 83| 94| 94| 88| 94| 82| 56| 83| 66| 80| 41| 86| 27| 11| 12| 6| 8| 8| €172.1M|
 6|177003| L. Modrić| 32|https://cdn.sofif...| Croatia| 91| 91| Real Madrid| €67M|€420K| Right| 4| 4| RCM| 10| 5'8|146lbs|77+3|77+3|77+3|85+3|85+3|87+3|86+3|88+3|86+3|81+3|79+3|71+3|71+3|71+3|79+3| 86| 72| 55| 93| 76| 90| 85| 78| 88| 93| 80| 72| 94| 79| 68| 89| 58| 82| 83| 82| 76| 13| 9| 7| 14| 9| €137.4M|
 7|176580| L. Suárez| 31|https://cdn.sofif...| Uruguay| 91| 91| FC Barcelona| €80M|€455K| Right| 4| 3| RS| 9| 6'0|190lbs|87+5|87+5|87+5|86+5|86+5|85+5|84+5|79+5|84+5|68+5|66+5|63+5|63+5|63

In [4]:
df.printSchema()

root
-- _c0: string (nullable = true)
-- ID: string (nullable = true)
-- Name: string (nullable = true)
-- Age: string (nullable = true)
-- Photo: string (nullable = true)
-- Nationality: string (nullable = true)
-- Overall: string (nullable = true)
-- Potential: string (nullable = true)
-- Club: string (nullable = true)
-- Value: string (nullable = true)
-- Wage: string (nullable = true)
-- Preferred Foot: string (nullable = true)
-- Weak Foot: string (nullable = true)
-- Skill Moves: string (nullable = true)
-- Position: string (nullable = true)
-- Jersey Number: string (nullable = true)
-- Height: string (nullable = true)
-- Weight: string (nullable = true)
-- LS: string (nullable = true)
-- ST: string (nullable = true)
-- RS: string (nullable = true)
-- LW: string (nullable = true)
-- RW: string (nullable = true)
-- CAM: string (nullable = true)
-- LM: string (nullable = true)
-- CM: string (nullable = true)
-- RM: string (nullable = true)
-- CDM: string (nullable = true)
-- LB: string (nullable = true)
-- LCB: string (nullable = true)
-- CB: string (nullable = true)
-- RCB: string (nullable = true)
-- RB: string (nullable = true)
-- Crossing: string (nullable = true)
-- Finishing: string (nullable = true)
-- HeadingAccuracy: string (nullable = true)
-- ShortPassing: string (nullable = true)
-- Volleys: string (nullable = true)
-- Dribbling: string (nullable = true)
-- Curve: string (nullable = true)
-- FKAccuracy: string (nullable = true)
-- LongPassing: string (nullable = true)
-- BallControl: string (nullable = true)
-- Acceleration: string (nullable = true)
-- SprintSpeed: string (nullable = true)
-- Balance: string (nullable = true)
-- ShotPower: string (nullable = true)
-- Jumping: string (nullable = true)
-- Stamina: string (nullable = true)
-- Strength: string (nullable = true)
-- LongShots: string (nullable = true)
-- Interceptions: string (nullable = true)
-- Penalties: string (nullable = true)
-- StandingTackle: string (nullable = true)
-- GKDiving: string (nullable = true)
-- GKHandling: string (nullable = true)
-- GKKicking: string (nullable = true)
-- GKPositioning: string (nullable = true)
-- GKReflexes: string (nullable = true)
-- Release Clause: string (nullable = true)

In [5]:
df.select('_c0','ID','Name','Age','Photo','Nationality','Overall','Potential','CLub','Value','Wage','Preferred Foot','Skill Moves','Position','Jersey Number','Height','Weight','Release Clause').show(10)

+---+------+-----------------+---+--------------------+-----------+-------+---------+-------------------+-------+-----+--------------+-----------+--------+-------------+------+------+--------------+
_c0| ID| Name|Age| Photo|Nationality|Overall|Potential| CLub| Value| Wage|Preferred Foot|Skill Moves|Position|Jersey Number|Height|Weight|Release Clause|
+---+------+-----------------+---+--------------------+-----------+-------+---------+-------------------+-------+-----+--------------+-----------+--------+-------------+------+------+--------------+
 0|158023| L. Messi| 31|https://cdn.sofif...| Argentina| 94| 94| FC Barcelona|€110.5M|€565K| Left| 4| RF| 10| 5'7|159lbs| €226.5M|
 1| 20801|Cristiano Ronaldo| 33|https://cdn.sofif...| Portugal| 94| 94| Juventus| €77M|€405K| Right| 5| ST| 7| 6'2|183lbs| €127.1M|
 2|190871| Neymar Jr| 26|https://cdn.sofif...| Brazil| 92| 93|Paris Saint-Germain|€118.5M|€290K| Right| 5| LW| 10| 5'9|150lbs| €228.1M|
 3|193080| De Gea| 27|https://cdn.sofif...| Spain| 91| 93| Manchester United| €72M|€260K| Right| 1| GK| 1| 6'4|168lbs| €138.6M|
 4|192985| K. De Bruyne| 27|https://cdn.sofif...| Belgium| 91| 92| Manchester City| €102M|€355K| Right| 4| RCM| 7| 5'11|154lbs| €196.4M|
 5|183277| E. Hazard| 27|https://cdn.sofif...| Belgium| 91| 91| Chelsea| €93M|€340K| Right| 4| LF| 10| 5'8|163lbs| €172.1M|
 6|177003| L. Modrić| 32|https://cdn.sofif...| Croatia| 91| 91| Real Madrid| €67M|€420K| Right| 4| RCM| 10| 5'8|146lbs| €137.4M|
 7|176580| L. Suárez| 31|https://cdn.sofif...| Uruguay| 91| 91| FC Barcelona| €80M|€455K| Right| 3| RS| 9| 6'0|190lbs| €164M|
 8|155862| Sergio Ramos| 32|https://cdn.sofif...| Spain| 91| 91| Real Madrid| €51M|€380K| Right| 3| RCB| 15| 6'0|181lbs| €104.6M|
 9|200389| J. Oblak| 25|https://cdn.sofif...| Slovenia| 90| 93| Atlético Madrid| €68M| €94K| Right| 1| GK| 1| 6'2|192lbs| €144.5M|
+---+------+-----------------+---+--------------------+-----------+-------+---------+-------------------+-------+-----+--------------+-----------+--------+-------------+------+------+--------------+
only showing top 10 rows

In [6]:
df.columns #Column Names
df.count() # Row Count
len(df.columns) #Column Count

Out[263]: 60

In [7]:
print("The Fifa has information of {} Players".format(df.count()))

The Fifa has information of 14962 Players

In [8]:
df=df.drop('Photo','Weak Foot','Balance','SlidingTackle')

In [9]:
df.describe()

Out[266]: DataFrame[summary: string, _c0: string, ID: string, Name: string, Age: string, Nationality: string, Overall: string, Potential: string, Club: string, Value: string, Wage: string, Preferred Foot: string, Skill Moves: string, Position: string, Jersey Number: string, Height: string, Weight: string, LS: string, ST: string, RS: string, LW: string, RW: string, CAM: string, LM: string, CM: string, RM: string, CDM: string, LB: string, LCB: string, CB: string, RCB: string, RB: string, Crossing: string, Finishing: string, HeadingAccuracy: string, ShortPassing: string, Volleys: string, Dribbling: string, Curve: string, FKAccuracy: string, LongPassing: string, BallControl: string, Acceleration: string, SprintSpeed: string, ShotPower: string, Jumping: string, Stamina: string, Strength: string, LongShots: string, Interceptions: string, Penalties: string, StandingTackle: string, GKDiving: string, GKHandling: string, GKKicking: string, GKPositioning: string, GKReflexes: string, Release Clause: string]

In [10]:
df = df.withColumnRenamed('_c0', 'c1')
df.columns

Out[267]: ['c1',
 'ID',
 'Name',
 'Age',
 'Nationality',
 'Overall',
 'Potential',
 'Club',
 'Value',
 'Wage',
 'Preferred Foot',
 'Skill Moves',
 'Position',
 'Jersey Number',
 'Height',
 'Weight',
 'LS',
 'ST',
 'RS',
 'LW',
 'RW',
 'CAM',
 'LM',
 'CM',
 'RM',
 'CDM',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Interceptions',
 'Penalties',
 'StandingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes',
 'Release Clause']

In [11]:
df.registerTempTable("fifa")

In [13]:
display(sqlContext.sql("SELECT Nationality, count(Name,ID,) \
                              FROM fifa \
                              Group By Nationality"))

Nationality,"count(Name, ID)"
Chad,1
Russia,63
Paraguay,60
Senegal,123
Sweden,295
Guyana,2
Philippines,1
Eritrea,2
Fiji,1
Turkey,252


In [14]:
display(sqlContext.sql("SELECT (Name, Age, Nationality, Club, Overall) \
                              FROM fifa \
                              where overall>=90 \
                              Order By 1 asc"))

"named_struct(Name, Name, Age, Age, Nationality, Nationality, Club, Club, Overall, Overall)"
"List(Cristiano Ronaldo, 33, Portugal, Juventus, 94)"
"List(D. Godín, 32, Uruguay, Atlético Madrid, 90)"
"List(David Silva, 32, Spain, Manchester City, 90)"
"List(De Gea, 27, Spain, Manchester United, 91)"
"List(E. Hazard, 27, Belgium, Chelsea, 91)"
"List(J. Oblak, 25, Slovenia, Atlético Madrid, 90)"
"List(K. De Bruyne, 27, Belgium, Manchester City, 91)"
"List(L. Messi, 31, Argentina, FC Barcelona, 94)"
"List(L. Modrić, 32, Croatia, Real Madrid, 91)"
"List(L. Suárez, 31, Uruguay, FC Barcelona, 91)"


In [15]:
display(sqlContext.sql("SELECT Position, Name, Overall, Club, Nationality, Value \
                                FROM fifa \
                                where Position='GK' \
                                Order By 3 desc"))

Position,Name,Overall,Club,Nationality,Value
GK,De Gea,91,Manchester United,Spain,€72M
GK,J. Oblak,90,Atlético Madrid,Slovenia,€68M
GK,T. Courtois,89,Real Madrid,Belgium,€53.5M
GK,M. Neuer,89,FC Bayern München,Germany,€38M
GK,M. ter Stegen,89,FC Barcelona,Germany,€58M
GK,G. Buffon,88,Paris Saint-Germain,Italy,€4M
GK,H. Lloris,88,Tottenham Hotspur,France,€36M
GK,S. Handanovič,88,Inter,Slovenia,€30M
GK,K. Navas,87,Real Madrid,Costa Rica,€30.5M
GK,Ederson,86,Manchester City,Brazil,€41.5M


In [16]:
display(sqlContext.sql("SELECT Position, Name, Overall, Club, Nationality, Value \
                                FROM fifa \
                                where Position='ST' or Position='LW' or Position='RW' or Position='RF' or position='LF'\
                                Order By 3 desc"))

Position,Name,Overall,Club,Nationality,Value
ST,Cristiano Ronaldo,94,Juventus,Portugal,€77M
RF,L. Messi,94,FC Barcelona,Argentina,€110.5M
LW,Neymar Jr,92,Paris Saint-Germain,Brazil,€118.5M
LF,E. Hazard,91,Chelsea,Belgium,€93M
ST,R. Lewandowski,90,FC Bayern München,Poland,€77M
ST,S. Agüero,89,Manchester City,Argentina,€64.5M
LF,P. Dybala,89,Juventus,Argentina,€89M
ST,H. Kane,89,Tottenham Hotspur,England,€83.5M
LW,Isco,88,Real Madrid,Spain,€73.5M
LW,Coutinho,88,FC Barcelona,Brazil,€69.5M


In [17]:
display(sqlContext.sql("SELECT Position, Name, Overall, Club, Nationality, Value \
                                FROM fifa \
                                where Position='CM' or Position='LM' or Position='RM' or Position='CDM'or Position='CAM'\
                                Order By 3 desc"))

Position,Name,Overall,Club,Nationality,Value
CDM,Sergio Busquets,89,FC Barcelona,Spain,€51.5M
CAM,A. Griezmann,89,Atlético Madrid,France,€78M
CAM,C. Eriksen,88,Tottenham Hotspur,Denmark,€73.5M
RM,K. Mbappé,88,Paris Saint-Germain,France,€81M
LM,P. Aubameyang,88,Arsenal,Gabon,€59M
RM,M. Salah,88,Liverpool,Egypt,€69.5M
CDM,Casemiro,88,Real Madrid,Brazil,€59.5M
CM,Thiago,86,FC Bayern München,Spain,€45.5M
LM,M. Reus,86,Borussia Dortmund,Germany,€43.5M
CAM,M. Özil,86,Arsenal,Germany,€43.5M


In [18]:
display(sqlContext.sql("SELECT Position, Name, Overall, Club, Nationality, Value \
                                FROM fifa \
                                where Position='CB' or Position='LB' or Position='RB' \
                                Order By 3 desc"))

Position,Name,Overall,Club,Nationality,Value
CB,D. Godín,90,Atlético Madrid,Uruguay,€44M
LB,Marcelo,88,Real Madrid,Brazil,€43M
CB,S. Umtiti,87,FC Barcelona,France,€57M
LB,Jordi Alba,87,FC Barcelona,Spain,€38M
RB,Azpilicueta,86,Chelsea,Spain,€35M
CB,M. Benatia,86,Juventus,Morocco,€30M
LB,Alex Sandro,86,Juventus,Brazil,€36.5M
LB,Filipe Luís,85,Atlético Madrid,Brazil,€21.5M
CB,N. Otamendi,85,Manchester City,Argentina,€28.5M
LB,D. Alaba,85,FC Bayern München,Austria,€38M


In [19]:
display(sqlContext.sql("SELECT Name,Age,Position,Overall,Club,Nationality \
                                FROM fifa \
                                where Age <=20\
                                Order By 3 desc"))

Name,Age,Position,Overall,Club,Nationality
M. Barrow,19,ST,68,Atalanta,Gambia
Kikas,19,ST,64,Os Belenenses,Portugal
J. Valencia,18,ST,60,Millonarios FC,Colombia
Raul Asencio,20,ST,65,Benevento,Spain
V. Damascan,19,ST,65,Torino,Moldova
You Ju An,19,ST,65,Suwon Samsung Bluewings,Korea Republic
A. Verda,19,ST,60,Querétaro,Mexico
J. Arp,18,ST,69,Hamburger SV,Germany
T. Awoniyi,20,ST,68,KAA Gent,Nigeria
W. Ponce,20,ST,65,CD Universidad de Concepción,Chile


In [20]:
display(sqlContext.sql("SELECT Name,Age,Position,Overall,Club,Nationality \
                                FROM fifa \
                                where Age >=30 and Overall<=85\
                                Order By 3 desc"))

Name,Age,Position,Overall,Club,Nationality
R. Simpson,33,ST,59,Milton Keynes Dons,England
R. O'Donovan,32,ST,66,Newcastle Jets,Republic of Ireland
R. Meggiorini,32,ST,74,Chievo Verona,Italy
Luimo Boas Santos,30,ST,79,Santos,Brazil
Jô,31,ST,77,Nagoya Grampus,Brazil
A. Gyan,32,ST,74,Kayserispor,Ghana
M. Sawa,35,ST,59,Kashiwa Reysol,Japan
S. Floccari,36,ST,74,SPAL,Italy
N. Kalinić,30,ST,79,Atlético Madrid,Croatia
M. Diop,30,ST,74,Beijing Renhe FC,Senegal


In [21]:
df.groupBy("Nationality").count().collect()

Out[277]: [Row(Nationality='Chad', count=1),
 Row(Nationality='Russia', count=63),
 Row(Nationality='Paraguay', count=60),
 Row(Nationality='Senegal', count=123),
 Row(Nationality='Sweden', count=295),
 Row(Nationality='Guyana', count=2),
 Row(Nationality='Philippines', count=1),
 Row(Nationality='Eritrea', count=2),
 Row(Nationality='Fiji', count=1),
 Row(Nationality='Turkey', count=252),
 Row(Nationality='Iraq', count=5),
 Row(Nationality='Germany', count=1030),
 Row(Nationality='St Kitts Nevis', count=2),
 Row(Nationality='Comoros', count=6),
 Row(Nationality='Afghanistan', count=2),
 Row(Nationality='Ivory Coast', count=82),
 Row(Nationality='Jordan', count=1),
 Row(Nationality='Rwanda', count=1),
 Row(Nationality='Sudan', count=3),
 Row(Nationality='France', count=787),
 Row(Nationality='Greece', count=91),
 Row(Nationality='Kosovo', count=29),
 Row(Nationality='Central African Rep.', count=3),
 Row(Nationality='DR Congo', count=48),
 Row(Nationality='Montserrat', count=4),
 Row(Nationality='Algeria', count=57),
 Row(Nationality='Togo', count=12),
 Row(Nationality='Equatorial Guinea', count=4),
 Row(Nationality='Slovakia', count=50),
 Row(Nationality='Argentina', count=859),
 Row(Nationality='Wales', count=102),
 Row(Nationality='Belgium', count=238),
 Row(Nationality='Angola', count=14),
 Row(Nationality='St Lucia', count=1),
 Row(Nationality='Ecuador', count=30),
 Row(Nationality='Qatar', count=1),
 Row(Nationality='Albania', count=31),
 Row(Nationality='Madagascar', count=12),
 Row(Nationality='Finland', count=50),
 Row(Nationality='New Caledonia', count=1),
 Row(Nationality='Ghana', count=106),
 Row(Nationality='Nicaragua', count=1),
 Row(Nationality='Peru', count=27),
 Row(Nationality='Benin', count=14),
 Row(Nationality='Sierra Leone', count=4),
 Row(Nationality='United States', count=301),
 Row(Nationality='Curacao', count=13),
 Row(Nationality='India', count=1),
 Row(Nationality='Belarus', count=4),
 Row(Nationality='Kuwait', count=1),
 Row(Nationality='Malta', count=1),
 Row(Nationality='Chile', count=310),
 Row(Nationality='Puerto Rico', count=1),
 Row(Nationality='Croatia', count=115),
 Row(Nationality='Burundi', count=2),
 Row(Nationality='Nigeria', count=102),
 Row(Nationality='Bolivia', count=5),
 Row(Nationality='Andorra', count=1),
 Row(Nationality='Gabon', count=14),
 Row(Nationality='Korea Republic', count=289),
 Row(Nationality='Italy', count=637),
 Row(Nationality='Suriname', count=4),
 Row(Nationality='Lithuania', count=7),
 Row(Nationality='Norway', count=257),
 Row(Nationality='Spain', count=1000),
 Row(Nationality='Cuba', count=4),
 Row(Nationality='Mauritania', count=4),
 Row(Nationality='Denmark', count=244),
 Row(Nationality='Niger', count=1),
 Row(Nationality='Barbados', count=3),
 Row(Nationality='Iran', count=16),
 Row(Nationality='Congo', count=19),
 Row(Nationality='Liechtenstein', count=2),
 Row(Nationality='Thailand', count=4),
 Row(Nationality='Morocco', count=81),
 Row(Nationality='Cape Verde', count=18),
 Row(Nationality='Panama', count=14),
 Row(Nationality='Hong Kong', count=2),
 Row(Nationality='Korea DPR', count=2),
 Row(Nationality='Ukraine', count=67),
 Row(Nationality='Venezuela', count=51),
 Row(Nationality='Iceland', count=34),
 Row(Nationality='Israel', count=14),
 Row(Nationality='Bosnia Herzegovina', count=54),
 Row(Nationality='Oman', count=1),
 Row(Nationality='Cyprus', count=5),
 Row(Nationality='Palestine', count=1),
 Row(Nationality='Uruguay', count=139),
 Row(Nationality='Mexico', count=309),
 Row(Nationality='FYR Macedonia', count=18),
 Row(Nationality='Montenegro', count=20),
 Row(Nationality='Zimbabwe', count=13),
 Row(Nationality='Estonia', count=11),
 Row(Nationality='Georgia', count=24),
 Row(Nationality='Guam', count=1),
 Row(Nationality='Guatemala', count=1),
 Row(Nationality='Libya', count=3),
 Row(Nationality='Azerbaijan', count=4),
 Row(Nationality='Grenada', count=1),
 Row(Nationality='Armenia', count=10),
 Row(Nationality='Tunisia', count=32),
 Row(Nation

In [22]:
df.groupBy("Club").count().collect()

Out[278]: [Row(Club='Palermo', count=28),
 Row(Club='Göztepe SK', count=25),
 Row(Club='CD Everton de Viña del Mar', count=21),
 Row(Club='Shonan Bellmare', count=17),
 Row(Club='Yeovil Town', count=22),
 Row(Club='Sagan Tosu', count=24),
 Row(Club='1. FC Union Berlin', count=23),
 Row(Club='Carpi', count=26),
 Row(Club='Puebla FC', count=25),
 Row(Club='Argentinos Juniors', count=26),
 Row(Club='SC Paderborn 07', count=22),
 Row(Club='Karlsruher SC', count=24),
 Row(Club='Cheltenham Town', count=19),
 Row(Club='SC Freiburg', count=27),
 Row(Club='San Lorenzo de Almagro', count=28),
 Row(Club='SpVgg Unterhaching', count=19),
 Row(Club='Universidad Católica', count=22),
 Row(Club='GFC Ajaccio', count=18),
 Row(Club='FC Luzern', count=24),
 Row(Club='AIK', count=19),
 Row(Club='SC Heerenveen', count=24),
 Row(Club='Sandefjord Fotball', count=18),
 Row(Club='Santos', count=19),
 Row(Club='BB Erzurumspor', count=22),
 Row(Club='CD Universidad de Concepción', count=24),
 Row(Club='CD Aves', count=25),
 Row(Club='GIF Sundsvall', count=20),
 Row(Club='América FC (Minas Gerais)', count=20),
 Row(Club='NAC Breda', count=26),
 Row(Club='América de Cali', count=21),
 Row(Club='Hibernian', count=22),
 Row(Club='Mansfield Town', count=19),
 Row(Club='Toronto FC', count=25),
 Row(Club='En Avant de Guingamp', count=24),
 Row(Club='Perth Glory', count=19),
 Row(Club='AC Ajaccio', count=18),
 Row(Club='SC Preußen Münster', count=21),
 Row(Club='Independiente', count=28),
 Row(Club='FC St. Gallen', count=19),
 Row(Club='Racing Club', count=24),
 Row(Club='Real Sporting de Gijón', count=28),
 Row(Club='Unión de Santa Fe', count=26),
 Row(Club='Birmingham City', count=27),
 Row(Club='Extremadura UD', count=29),
 Row(Club='Portimonense SC', count=27),
 Row(Club='Swansea City', count=28),
 Row(Club='Urawa Red Diamonds', count=22),
 Row(Club='Belgrano de Córdoba', count=23),
 Row(Club='Hellas Verona', count=27),
 Row(Club='Dalkurd FF', count=17),
 Row(Club='Deportivo Toluca', count=22),
 Row(Club='Atiker Konyaspor', count=24),
 Row(Club='Sanfrecce Hiroshima', count=21),
 Row(Club='1. FC Magdeburg', count=23),
 Row(Club='Chicago Fire', count=21),
 Row(Club='Club América', count=30),
 Row(Club='Al Taawoun', count=20),
 Row(Club='SG Dynamo Dresden', count=24),
 Row(Club='Club Deportes Temuco', count=19),
 Row(Club='Botafogo', count=19),
 Row(Club='Sunderland', count=23),
 Row(Club='FC Seoul', count=26),
 Row(Club='Unión Española', count=21),
 Row(Club='Vejle Boldklub', count=19),
 Row(Club='Medipol Başakşehir FK', count=23),
 Row(Club='PFC CSKA Moscow', count=27),
 Row(Club='Envigado FC', count=20),
 Row(Club='Hamilton Academical FC', count=14),
 Row(Club='VfL Osnabrück', count=21),
 Row(Club='Notts County', count=19),
 Row(Club='Atlanta United', count=23),
 Row(Club='Górnik Zabrze', count=19),
 Row(Club='Sport Club do Recife', count=18),
 Row(Club='Pohang Steelers', count=22),
 Row(Club='CD Huachipato', count=16),
 Row(Club='Derry City', count=5),
 Row(Club='FC Red Bull Salzburg', count=25),
 Row(Club='Shanghai Greenland Shenhua FC', count=19),
 Row(Club='Bolton Wanderers', count=24),
 Row(Club='Rio Ave FC', count=26),
 Row(Club='Nîmes Olympique', count=23),
 Row(Club='Al Hazem', count=23),
 Row(Club='Korona Kielce', count=20),
 Row(Club='Milton Keynes Dons', count=24),
 Row(Club='La Berrichonne de Châteauroux', count=20),
 Row(Club='Real Valladolid CF', count=29),
 Row(Club='Sporting Lokeren', count=25),
 Row(Club='Minnesota United FC', count=29),
 Row(Club='Moreirense FC', count=22),
 Row(Club='San Martin de Tucumán', count=25),
 Row(Club='Junior FC', count=26),
 Row(Club='Wolfsberger AC', count=17),
 Row(Club='Macclesfield Town', count=16),
 Row(Club='Club Atlas', count=25),
 Row(Club='Boavista FC', count=24),
 Row(Club='Club Necaxa', count=26),
 Row(Club='Brisbane Roar', count=16),
 Row(Club='Bradford City', count=22),
 Row(Club='Manchester United', count=33),
 Row(Club='Antalyaspor', count=24),
 Row(Club='Al Fateh', count=19),
 Row(Club='Central 

In [23]:
df.groupBy("Club","Name").agg(count(lit(1)).alias("cnt")).filter(col("cnt")>1).show()

+--------------------+-------------+---+
 Club| Name|cnt|
+--------------------+-------------+---+
 Roma|L. Pellegrini| 2|
 Colo-Colo| D. Pérez| 2|
 Unión Española| L. Pavez| 2|
 Atlético Nacional| J. Ramírez| 2|
 Albacete BP| Caro| 2|
 FC Carl Zeiss Jena| F. Brügmann| 2|
 Rochdale| J. Williams| 2|
Chongqing Dangdai...| Fernandinho| 2|
 Lokomotiv Moscow| A. Miranchuk| 2|
 Sagan Tosu| Y. Takahashi| 2|
 Al Hilal| M. Fallatah| 2|
+--------------------+-------------+---+

In [24]:
df.show(2)

+---+------+-----------------+---+-----------+-------+---------+------------+-------+-----+--------------+-----------+--------+-------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+---------+-------+-------+--------+---------+-------------+---------+--------------+--------+----------+---------+-------------+----------+--------------+
 c1| ID| Name|Age|Nationality|Overall|Potential| Club| Value| Wage|Preferred Foot|Skill Moves|Position|Jersey Number|Height|Weight| LS| ST| RS| LW| RW| CAM| LM| CM| RM| CDM| LB| LCB| CB| RCB| RB|Crossing|Finishing|HeadingAccuracy|ShortPassing|Volleys|Dribbling|Curve|FKAccuracy|LongPassing|BallControl|Acceleration|SprintSpeed|ShotPower|Jumping|Stamina|Strength|LongShots|Interceptions|Penalties|StandingTackle|GKDiving|GKHandling|GKKicking|GKPositioning|GKReflexes|Release Clause|
+---+------+-----------------+---+-----------+-------+---------+------------+-------+-----+--------------+-----------+--------+-------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+---------+-------+-------+--------+---------+-------------+---------+--------------+--------+----------+---------+-------------+----------+--------------+
 0|158023| L. Messi| 31| Argentina| 94| 94|FC Barcelona|€110.5M|€565K| Left| 4| RF| 10| 5'7|159lbs|88+2|88+2|88+2|92+2|92+2|93+2|91+2|84+2|91+2|61+2|59+2|47+2|47+2|47+2|59+2| 84| 95| 70| 90| 86| 97| 93| 94| 87| 96| 91| 86| 85| 68| 72| 59| 94| 22| 75| 28| 6| 11| 15| 14| 8| €226.5M|
 1| 20801|Cristiano Ronaldo| 33| Portugal| 94| 94| Juventus| €77M|€405K| Right| 5| ST| 7| 6'2|183lbs|91+3|91+3|91+3|89+3|89+3|88+3|88+3|81+3|88+3|61+3|61+3|53+3|53+3|53+3|61+3| 84| 94| 89| 81| 87| 88| 81| 76| 77| 94| 89| 91| 95| 95| 88| 79| 93| 29| 85| 31| 7| 11| 15| 14| 11| €127.1M|
+---+------+-----------------+---+-----------+-------+---------+------------+-------+-----+--------------+-----------+--------+-------------+------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+--------+---------+---------------+------------+-------+---------+-----+----------+-----------+-----------+------------+-----------+---------+-------+-------+--------+---------+-------------+---------+--------------+--------+----------+---------+-------------+----------+--------------+
only showing top 2 rows

In [25]:
from pyspark.sql.functions import mean
from pyspark.sql.functions import max,min
df.agg({"Age":"avg"}).show()
display(df.select(max("Age"),min("Age"))).show()

max(Age),min(Age)
45,16


In [26]:
display(sqlContext.sql("SELECT Name,Age,Position,Overall,Club,Nationality \
                                FROM fifa \
                                where Age =16\
                                Order By 5 desc"))

Name,Age,Position,Overall,Club,Nationality
Y. Roemer,16,ST,59,VVV-Venlo,Netherlands
A. Taoui,16,ST,64,Toulouse Football Club,France
Pelayo Morilla,16,RM,63,Real Sporting de Gijón,Spain
Y. Verschaeren,16,CAM,62,RSC Anderlecht,Belgium
J. Lahne,16,ST,59,IF Brommapojkarna,Sweden
Guerrero,16,CM,63,CF Rayo Majadahonda,Spain
W. Geubbels,16,ST,64,AS Monaco,France
Y. Begraoui,16,ST,59,AJ Auxerre,France


In [27]:
display(sqlContext.sql("SELECT Name,Age,Position,Overall,Club,Nationality \
                                FROM fifa \
                                where Age =45\
                                Order By 5 desc"))

Name,Age,Position,Overall,Club,Nationality
O. Pérez,45,GK,71,Pachuca,Mexico


In [28]:
display(sqlContext.sql("SELECT Club, Name,potential \
                              FROM fifa \
                              where potential>=90"))

Club,Name,potential
FC Barcelona,L. Messi,94
Juventus,Cristiano Ronaldo,94
Paris Saint-Germain,Neymar Jr,93
Manchester United,De Gea,93
Manchester City,K. De Bruyne,92
Chelsea,E. Hazard,91
Real Madrid,L. Modrić,91
FC Barcelona,L. Suárez,91
Real Madrid,Sergio Ramos,91
Atlético Madrid,J. Oblak,93


In [30]:
count = [item[1] for item in df.groupBy("Club").count().collect()]
year = [item[0] for item in df.groupBy("Club").count().collect()]
Players_Per_Club = {"Players":count, "Club" : year}
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
Players_Per_Club = pd.DataFrame(Players_Per_Club)
display(Players_Per_Club)

Players,Club
28,Palermo
25,Göztepe SK
21,CD Everton de Viña del Mar
17,Shonan Bellmare
22,Yeovil Town
24,Sagan Tosu
23,1. FC Union Berlin
26,Carpi
25,Puebla FC
26,Argentinos Juniors


In [31]:
df.select(max("Overall")).show()
df.select(min("Overall")).show()

+------------+
max(Overall)|
+------------+
 94|
+------------+

+------------+
min(Overall)|
+------------+
 58|
+------------+

In [33]:
# Use the Spark CSV datasource with options specifying:
# - First line of file is a header
# - Automatically infer the schema of the data
data = spark.read.csv("/FileStore/tables/Fifa_2017-ec2ed.csv", header="true", inferSchema="true")
data.cache() # Cache data for faster reuse

Out[284]: DataFrame[_c0: int, ID: int, Name: string, Age: int, Photo: string, Nationality: string, Overall: int, Potential: int, Club: string, Value: string, Wage: string, Preferred Foot: string, Weak Foot: int, Skill Moves: int, Position: string, Jersey Number: int, Height: string, Weight: string, LS: string, ST: string, RS: string, LW: string, RW: string, CAM: string, LM: string, CM: string, RM: string, CDM: string, LB: string, LCB: string, CB: string, RCB: string, RB: string, Crossing: int, Finishing: int, HeadingAccuracy: int, ShortPassing: int, Volleys: int, Dribbling: int, Curve: int, FKAccuracy: int, LongPassing: int, BallControl: int, Acceleration: int, SprintSpeed: int, Balance: int, ShotPower: int, Jumping: int, Stamina: int, Strength: int, LongShots: int, Interceptions: int, Penalties: int, StandingTackle: int, GKDiving: int, GKHandling: int, GKKicking: int, GKPositioning: int, GKReflexes: int, Release Clause: string]

In [34]:
display(data)

_c0,ID,Name,Age,Photo,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,Weak Foot,Skill Moves,Position,Jersey Number,Height,Weight,LS,ST,RS,LW,RW,CAM,LM,CM,RM,CDM,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Interceptions,Penalties,StandingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,94,94,FC Barcelona,€110.5M,€565K,Left,4,4,RF,10,5'7,159lbs,88+2,88+2,88+2,92+2,92+2,93+2,91+2,84+2,91+2,61+2,59+2,47+2,47+2,47+2,59+2,84,95,70,90,86,97,93,94,87,96,91,86,95,85,68,72,59,94,22,75,28,6,11,15,14,8,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,94,94,Juventus,€77M,€405K,Right,4,5,ST,7,6'2,183lbs,91+3,91+3,91+3,89+3,89+3,88+3,88+3,81+3,88+3,61+3,61+3,53+3,53+3,53+3,61+3,84,94,89,81,87,88,81,76,77,94,89,91,70,95,95,88,79,93,29,85,31,7,11,15,14,11,€127.1M
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,Right,5,5,LW,10,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,88+3,81+3,88+3,60+3,60+3,47+3,47+3,47+3,60+3,79,87,62,84,84,96,88,87,78,95,94,90,84,80,61,81,49,82,36,81,24,9,9,15,15,11,€228.1M
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,91,92,Manchester City,€102M,€355K,Right,5,4,RCM,7,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,88+3,88+3,87+3,88+3,77+3,73+3,66+3,66+3,66+3,73+3,93,82,55,92,82,86,85,83,91,91,78,76,77,91,63,90,75,91,61,79,58,15,13,5,10,13,€196.4M
5,183277,E. Hazard,27,https://cdn.sofifa.org/players/4/19/183277.png,Belgium,91,91,Chelsea,€93M,€340K,Right,4,4,LF,10,5'8,163lbs,83+3,83+3,83+3,89+3,89+3,89+3,89+3,82+3,89+3,63+3,60+3,49+3,49+3,49+3,60+3,81,84,61,89,80,95,83,79,83,94,94,88,94,82,56,83,66,80,41,86,27,11,12,6,8,8,€172.1M
6,177003,L. Modrić,32,https://cdn.sofifa.org/players/4/19/177003.png,Croatia,91,91,Real Madrid,€67M,€420K,Right,4,4,RCM,10,5'8,146lbs,77+3,77+3,77+3,85+3,85+3,87+3,86+3,88+3,86+3,81+3,79+3,71+3,71+3,71+3,79+3,86,72,55,93,76,90,85,78,88,93,80,72,94,79,68,89,58,82,83,82,76,13,9,7,14,9,€137.4M
7,176580,L. Suárez,31,https://cdn.sofifa.org/players/4/19/176580.png,Uruguay,91,91,FC Barcelona,€80M,€455K,Right,4,3,RS,9,6'0,190lbs,87+5,87+5,87+5,86+5,86+5,85+5,84+5,79+5,84+5,68+5,66+5,63+5,63+5,63+5,66+5,77,93,77,82,88,87,86,84,64,90,86,75,83,86,69,90,83,85,41,85,45,27,25,31,33,37,€164M
8,155862,Sergio Ramos,32,https://cdn.sofifa.org/players/4/19/155862.png,Spain,91,91,Real Madrid,€51M,€380K,Right,3,3,RCB,15,6'0,181lbs,73+3,73+3,73+3,70+3,70+3,71+3,72+3,75+3,72+3,84+3,84+3,87+3,87+3,87+3,84+3,66,60,91,78,66,63,74,72,77,84,76,75,66,79,93,84,83,59,90,75,92,11,8,9,7,11,€104.6M
10,188545,R. Lewandowski,29,https://cdn.sofifa.org/players/4/19/188545.png,Poland,90,90,FC Bayern München,€77M,€205K,Right,4,4,ST,9,6'0,176lbs,87+3,87+3,87+3,83+3,83+3,83+3,81+3,77+3,81+3,62+3,58+3,57+3,57+3,57+3,58+3,62,91,85,83,89,85,77,86,65,89,77,78,78,88,84,78,84,84,39,88,42,15,6,12,8,10,€127.1M
11,182521,T. Kroos,28,https://cdn.sofifa.org/players/4/19/182521.png,Germany,90,90,Real Madrid,€76.5M,€355K,Right,5,3,LCM,8,6'0,168lbs,78+3,78+3,78+3,81+3,81+3,84+3,82+3,86+3,82+3,82+3,77+3,72+3,72+3,72+3,77+3,88,76,54,92,82,81,86,84,93,90,64,62,71,87,30,75,73,92,82,73,79,10,11,13,7,10,€156.8M


In [35]:
from pyspark.sql.functions import col

data = data.dropna() # drop rows with missing values
exprs = [col(column).alias(column.replace(' ', '0')) for column in data.columns]
vdata = data.select(*exprs).selectExpr("Age", "Overall")
display(vdata)

Age,Overall
31,94
33,94
26,92
27,91
27,91
32,91
31,91
32,91
29,90
28,90


In [36]:
#(For the model we are doing supervised learning for feature we are using "age" and "Overall )

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler


stages = []
assembler = VectorAssembler(inputCols=["Age"], outputCol="features")
stages += [assembler]
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(vdata)
dataset = pipelineModel.transform(vdata)
# Keep relevant columns
selectedcols = ["features"]

In [37]:
display(dataset)

Age,Overall,features
31,94,"List(1, 1, List(), List(31.0))"
33,94,"List(1, 1, List(), List(33.0))"
26,92,"List(1, 1, List(), List(26.0))"
27,91,"List(1, 1, List(), List(27.0))"
27,91,"List(1, 1, List(), List(27.0))"
32,91,"List(1, 1, List(), List(32.0))"
31,91,"List(1, 1, List(), List(31.0))"
32,91,"List(1, 1, List(), List(32.0))"
29,90,"List(1, 1, List(), List(29.0))"
28,90,"List(1, 1, List(), List(28.0))"


In [38]:
# Import LinearRegression class
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
lr = LinearRegression()

In [39]:
# Fit 2 models, using different regularization parameters
lr = LinearRegression(featuresCol = 'features', labelCol='Age', maxIter=10, regParam=0.3, elasticNetParam=0.7)
modelA = lr.fit(dataset, {lr.regParam:0.3})
modelB = lr.fit(dataset, {lr.regParam:0.7})
print(">>>> ModelA intercept: %r, coefficient: %r" % (modelA.intercept, modelA.coefficients[0]))
print(">>>> ModelB intercept: %r, coefficient: %r" % (modelB.intercept, modelB.coefficients[0]))

>>>> ModelA intercept: 1.7307207321005784, coefficient: 0.9327732857746949
>>>> ModelB intercept: 3.9325981150445934, coefficient: 0.8472453442432601

In [40]:
# Make predictions
predictionsA = modelA.transform(dataset)
display(predictionsA)

Age,Overall,features,prediction
31,94,"List(1, 1, List(), List(31.0))",30.64669259111612
33,94,"List(1, 1, List(), List(33.0))",32.51223916266551
26,92,"List(1, 1, List(), List(26.0))",25.982826162242645
27,91,"List(1, 1, List(), List(27.0))",26.91559944801734
27,91,"List(1, 1, List(), List(27.0))",26.91559944801734
32,91,"List(1, 1, List(), List(32.0))",31.579465876890815
31,91,"List(1, 1, List(), List(31.0))",30.64669259111612
32,91,"List(1, 1, List(), List(32.0))",31.579465876890815
29,90,"List(1, 1, List(), List(29.0))",28.78114601956673
28,90,"List(1, 1, List(), List(28.0))",27.848372733792036


In [41]:
predictionsB = modelB.transform(dataset)
display(predictionsB)

Age,Overall,features,prediction
31,94,"List(1, 1, List(), List(31.0))",30.19720378658566
33,94,"List(1, 1, List(), List(33.0))",31.89169447507218
26,92,"List(1, 1, List(), List(26.0))",25.96097706536936
27,91,"List(1, 1, List(), List(27.0))",26.80822240961262
27,91,"List(1, 1, List(), List(27.0))",26.80822240961262
32,91,"List(1, 1, List(), List(32.0))",31.04444913082892
31,91,"List(1, 1, List(), List(31.0))",30.19720378658566
32,91,"List(1, 1, List(), List(32.0))",31.04444913082892
29,90,"List(1, 1, List(), List(29.0))",28.50271309809914
28,90,"List(1, 1, List(), List(28.0))",27.655467753855877


In [42]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol = 'Age', predictionCol='prediction' , metricName="rmse")
RMSE = evaluator.evaluate(predictionsA)
print("ModelA: Root Mean Squared Error = " + str(RMSE))

predictionsB = modelB.transform(dataset)
RMSE = evaluator.evaluate(predictionsB)
print("ModelB: Root Mean Squared Error = " + str(RMSE))


ModelA: Root Mean Squared Error = 0.2939495957196425
ModelB: Root Mean Squared Error = 0.6679215222909886

In [43]:
# Import numpy, pandas, and ggplot
import numpy as np
from pandas import *
import matplotlib.pyplot as plt

# Create Python DataFrame
pop = dataset.rdd.map(lambda p: (p.features[0])).collect()

predA = predictionsA.select("prediction").rdd.map(lambda r: r[0]).collect()
predB = predictionsB.select("prediction").rdd.map(lambda r: r[0]).collect()

# Create a pandas DataFrame
pydf = DataFrame({'pop':pop,'predictionA':predA, 'predictionB':predB})
display(pydf)
# Visualizing the Model
# Create scatter plot and two regression models (scaling exponential) using Matplotlib
fig, ax = plt.subplots()
ax.loglog(x,y)
ax.scatter(x, y)
plt.xlim(1.0e5, 1.0e7)
plt.ylim(5.0e1, 1.0e3)
plt.title('Linear Models Visual Representation')
ax.plot(pop, predA, '.r-')
ax.plot(pop, predB, '.g-')
display(fig)

pop,predictionA,predictionB
31.0,30.64669259111612,30.19720378658566
33.0,32.51223916266551,31.89169447507218
26.0,25.982826162242645,25.96097706536936
27.0,26.91559944801734,26.80822240961262
27.0,26.91559944801734,26.80822240961262
32.0,31.579465876890815,31.04444913082892
31.0,30.64669259111612,30.19720378658566
32.0,31.579465876890815,31.04444913082892
29.0,28.78114601956673,28.50271309809914
28.0,27.848372733792036,27.655467753855877
